In [17]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import os # We'll use this to manage the directory for our files

In [12]:
# --- Scalability Parameters ---
NUM_CUSTOMERS = 200000 
MAX_BILLS_PER_CUSTOMER = 8
MAX_USAGE_RECORDS_PER_SUBSCRIPTION = 20

### Table 1: Plans

In [6]:
def create_plans_table():
    plans_data = {
        'plan_id': [101, 102, 103, 104, 105, 106, 107],
        'plan_name': ['Basic Talk', 'Data Pro', 'Unlimited Premium', 'Family Plan', 'Business Pro', 'Prepaid Basic', 'IoT Connect'],
        'monthly_cost_usd': [19.99, 49.99, 79.99, 99.99, 129.99, 15.00, 5.00],
        'data_gb': [2, 20, 100, 50, 150, 5, 0.5],
        'voice_minutes': [500, 1000, np.inf, np.inf, np.inf, 200, 0],
        'sms_count': [500, 1000, np.inf, np.inf, np.inf, 100, 0]
    }
    return pd.DataFrame(plans_data)

plans_df = create_plans_table()
print("Plans Table created. Records:", len(plans_df))

Plans Table created. Records: 7


### Table 2: Customers

In [4]:
def create_customers_table(num_customers):
    customer_ids = np.arange(100001, 100001 + num_customers)
    first_names = ['John', 'Jane', 'Peter', 'Mary', 'Robert', 'Patricia', 'Michael', 'Linda', 'William', 'Elizabeth']
    last_names = ['Smith', 'Johnson', 'Williams', 'Brown', 'Jones', 'Garcia', 'Miller', 'Davis', 'Rodriguez', 'Martinez']
    
    data = {
        'customer_id': customer_ids,
        'first_name': np.random.choice(first_names, num_customers),
        'last_name': np.random.choice(last_names, num_customers),
        'email': [f'user_{i}@example.com' for i in range(num_customers)],
        'signup_date': [datetime.now() - timedelta(days=random.randint(1, 1500)) for _ in range(num_customers)],
        'city': np.random.choice(['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix', 'San Francisco', 'Miami'], num_customers)
    }
    return pd.DataFrame(data)

customers_df = create_customers_table(NUM_CUSTOMERS)
print("Customers Table created. Records:", len(customers_df))

Customers Table created. Records: 100000


### Table 3: Subscription

In [7]:
def create_subscriptions_table(customers_df, plans_df):
    subscription_data = []
    
    plan_ids = plans_df['plan_id'].to_list()
    
    for _, customer_row in customers_df.iterrows():
        customer_id = customer_row['customer_id']
        plan_id = random.choice(plan_ids)
        
        subscription_date = customer_row['signup_date']
        start_date = subscription_date + timedelta(days=random.randint(0, 30))
        
        subscription_data.append({
            'subscription_id': len(subscription_data) + 1,
            'customer_id': customer_id,
            'plan_id': plan_id,
            'start_date': start_date,
            'is_active': random.choice([True, True, True, False])
        })
    
    return pd.DataFrame(subscription_data)

subscriptions_df = create_subscriptions_table(customers_df, plans_df)
print("Customer Subscriptions Table created. Records:", len(subscriptions_df))

Customer Subscriptions Table created. Records: 100000


### Table 4: Bills

In [10]:
def create_bills_table(subscriptions_df, plans_df):
    bills_data = []
    bill_id = 1
    
    # Pre-calculate plan costs for efficiency
    plan_cost_map = plans_df.set_index('plan_id')['monthly_cost_usd'].to_dict()
    
    for _, sub in subscriptions_df.iterrows():
        if sub['is_active']:
            num_bills = random.randint(1, MAX_BILLS_PER_CUSTOMER)
            
            for i in range(num_bills):
                bill_date = sub['start_date'] + timedelta(days=30 * (i+1))
                
                plan_cost = plan_cost_map.get(sub['plan_id'], 0)
                taxes = round(plan_cost * 0.1, 2)
                extra_charges = round(random.uniform(0, 5), 2)
                total_amount = round(plan_cost + taxes + extra_charges, 2)
                
                is_paid = random.choice([True] * 4 + [False]) # 80% paid
                payment_date = bill_date + timedelta(days=random.randint(1, 10)) if is_paid else None
                
                bills_data.append({
                    'bill_id': bill_id,
                    'subscription_id': sub['subscription_id'],
                    'bill_date': bill_date,
                    'amount_due_usd': total_amount,
                    'is_paid': is_paid,
                    'payment_date': payment_date
                })
                bill_id += 1
                
    return pd.DataFrame(bills_data)

bills_df = create_bills_table(subscriptions_df, plans_df)
print("Bills Table created. Records:", len(bills_df))

Bills Table created. Records: 486680


### Table 5: Usage_Details

In [14]:
def create_usage_details_table(subscriptions_df):
    usage_data = []
    usage_id = 1
    
    active_subscriptions = subscriptions_df[subscriptions_df['is_active']]
    
    for _, sub in active_subscriptions.iterrows():
        num_usage_records = random.randint(1, MAX_USAGE_RECORDS_PER_SUBSCRIPTION)
        
        for _ in range(num_usage_records):
            usage_date = sub['start_date'] + timedelta(days=random.randint(1, (MAX_BILLS_PER_CUSTOMER * 30)))
            
            usage_type = random.choice(['voice', 'data', 'sms'])
            
            if usage_type == 'voice':
                value = random.randint(1, 60)
                unit = 'minutes'
            elif usage_type == 'data':
                value = random.uniform(0.01, 5.0)
                unit = 'gb'
            else:
                value = random.randint(1, 10)
                unit = 'messages'
            
            usage_data.append({
                'usage_id': usage_id,
                'subscription_id': sub['subscription_id'],
                'usage_date': usage_date,
                'usage_type': usage_type,
                'usage_value': round(value, 2),
                'usage_unit': unit,
                'cost_usd': round(random.uniform(0, 2), 2) # Simulate some overage or specific service cost
            })
            usage_id += 1
            
    return pd.DataFrame(usage_data)

usage_df = create_usage_details_table(subscriptions_df)
print("Usage Details Table created. Records:", len(usage_df))

Usage Details Table created. Records: 787563


##  Verify the Total Number of Records

In [16]:
# --- Verification ---
total_records = len(plans_df) + len(customers_df) + len(subscriptions_df) + len(bills_df) + len(usage_df)

print("\n" + "="*50)
print("Final Dataset Creation Summary")
print("="*50)
print(f"Plans Table Records:            {len(plans_df):,}")
print(f"Customers Table Records:        {len(customers_df):,}")
print(f"Subscriptions Table Records:    {len(subscriptions_df):,}")
print(f"Bills Table Records:            {len(bills_df):,}")
print(f"Usage Details Table Records:    {len(usage_df):,}")
print("-" * 50)
print(f"Total Records Across All Tables: {total_records:,}")
print("="*50)

# Display a sample of the new table
print("\nSample of Usage Details Table:")
print(usage_df.sample(5))

print("\nSample of Bills Table:")
print(bills_df.sample(5))


Final Dataset Creation Summary
Plans Table Records:            7
Customers Table Records:        100,000
Subscriptions Table Records:    100,000
Bills Table Records:            486,680
Usage Details Table Records:    787,563
--------------------------------------------------
Total Records Across All Tables: 1,474,250

Sample of Usage Details Table:
        usage_id  subscription_id                 usage_date usage_type  \
556435    556436            70655 2022-01-02 15:05:46.994512      voice   
261120    261121            32981 2021-12-22 15:05:46.936647        sms   
216232    216233            27254 2024-08-15 15:05:46.927420       data   
538042    538043            68268 2024-01-21 15:05:46.987601      voice   
135317    135318            16990 2022-07-15 15:05:46.908481       data   

        usage_value usage_unit  cost_usd  
556435        37.00    minutes      0.46  
261120         3.00   messages      0.67  
216232         0.29         gb      1.16  
538042        17.00    mi

### Export DataFrames to CSV Files

In [18]:
# --- Export to CSV Files ---

# Create a directory to store the CSV files if it doesn't exist
output_dir = 'telecom_data'
os.makedirs(output_dir, exist_ok=True)

# Define file paths
plans_file = os.path.join(output_dir, 'plans.csv')
customers_file = os.path.join(output_dir, 'customers.csv')
subscriptions_file = os.path.join(output_dir, 'customer_subscriptions.csv')
bills_file = os.path.join(output_dir, 'bills.csv')
usage_file = os.path.join(output_dir, 'usage_details.csv')

# Export each DataFrame to its respective CSV file
# We use index=False to prevent pandas from writing the DataFrame index as a column
plans_df.to_csv(plans_file, index=False)
customers_df.to_csv(customers_file, index=False)
subscriptions_df.to_csv(subscriptions_file, index=False)
bills_df.to_csv(bills_file, index=False)
usage_df.to_csv(usage_file, index=False)

print("\n" + "="*50)
print("Export Complete")
print("="*50)
print(f"Data has been successfully exported to the '{output_dir}' directory.")
print(f"  - Plans: {plans_file}")
print(f"  - Customers: {customers_file}")
print(f"  - Subscriptions: {subscriptions_file}")
print(f"  - Bills: {bills_file}")
print(f"  - Usage Details: {usage_file}")


Export Complete
Data has been successfully exported to the 'telecom_data' directory.
  - Plans: telecom_data\plans.csv
  - Customers: telecom_data\customers.csv
  - Subscriptions: telecom_data\customer_subscriptions.csv
  - Bills: telecom_data\bills.csv
  - Usage Details: telecom_data\usage_details.csv
